# Key Idea  

## Client Side
- DPoP in headers when making request
- Signature is inside DPoP
- Signature = headers, claims, client private key
- headers contains alg, typ, jwk (public key in jwk format including e, kty, n in this exact order)
- claims contains jti, iat, exp (short live such as 15 to 30 seconds), htm, htu, client id (optional)

## Server Side
- Access Token = claims, server private key
- claims = jti, iat, exp (mid live such as 10 to 30 minutes), client id (optional), cnf.jkt (jwk from client's public key converting to thumbprint)
- Refresh Token = claims, server private key
- claims = jti, iat, exp (long live such as 24 hours or more), client id (optional)

In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import time

In [24]:
from package.jwt_management.data_models.base_models import JWK
from package.jwt_management.data_models.server_models import ServerSignature, JKT
from package.jwt_management.data_models.client_models import ClientHeaders, ClientClaims, ClientSignature

In [25]:
from package.keypair_management import KeyPairManagement
km = KeyPairManagement(directory='./client_keypair')
skm = KeyPairManagement(directory='./server_keypair')
km.generate_keypairs()

Directory './client_keypair' already exists.


In [26]:
DPOP_LIVE = 15
ACCESS_TOKEN_LIVE = 60*10
REFRESH_TOKEN_LIVE = 60*24*1

In [27]:
from package.ezorm.db_management import delete_database, create_tables, delete_tables

In [28]:
delete_database()

In [29]:
from package.database_management.data_models import DPoPModel, AccessTokenModel, RefreshTokenModel, UserModel, CodeModel
from package.ezorm.crud import Create, Read, Update, Delete

In [51]:
tables = [DPoPModel, AccessTokenModel, RefreshTokenModel, UserModel, CodeModel]
delete_tables(tables)
create_tables(tables)

Model: dpopmodel deleted successfully
Model: accesstokenmodel deleted successfully
Model: refreshtokenmodel deleted successfully
Model: usermodel deleted successfully
Model: codemodel deleted successfully
All tables deleted successfully
Model: dpopmodel created successfully
Model: accesstokenmodel created successfully
Model: refreshtokenmodel created successfully
Model: usermodel created successfully
Model: codemodel created successfully
All tables created successfully


# Client

In [52]:
jti = "test"
iat = int(time.time())
exp = iat + DPOP_LIVE
htm = "POST"
htu = "www.testapi.com/api/token"
client_id = "555"

In [53]:
ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)

ClientClaims(jti='test', iat=1733578607, exp=1733578622, htm='POST', htu='www.testapi.com/api/token', client_id='555')

In [54]:
ClientHeaders.from_key(key=km.load_public_key_from_pem())

ClientHeaders(typ=None, alg=None, jwk=JWK(e='AQAB', kty='RSA', n='nOC6RY5CimucaX9HjZFJPyOK-WNXGTAA_5zJgB6AAacSty5OiGBBx4ltub9YNuEaqNiSYctknZBq5JIe-lpzd2z6FcR6J45urCaKpZ6OhKaOecMWPvA91bb0uwp_W5HcH__cOKqKngV2m2st0p7VpS-UF2n8sFOsDdgBXQW3dvn73GgyWTy7DggEalJLwMYFkqQQ4LeyzGId_G3jkxFIZDSKFhfIJsrFK8mPFcFOU-e85hpDbx_3mjwwqqeYLi9YRnniRczzKFbKvrOATgYd96ihnpTC2SNj7CGHkzvJ7N2baqJPXHl2OremFofjUxZLgWuo9dWiLDTUTeT_nKHRFQ'))

In [55]:
c_sig = ClientSignature(
    headers=ClientHeaders(typ="dpop+jwt", alg="RS256", jwk=JWK.from_key(key=km.load_public_key_from_pem())),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)
)
signature = c_sig.sign(key=km.load_private_key_from_pem())

In [56]:
c_sig.headers.jwk.to_thumbprint()

'i450Qle7gG_lVTikvKHzb7ZRI3xUalVRMJ8dbCWL0vQ'

In [57]:
headers = c_sig.headers.model_dump()

In [58]:
headers.pop('typ')

'dpop+jwt'

In [59]:
c_sig.headers = ClientHeaders(**headers)

In [60]:
c_sig

ClientSignature(headers=ClientHeaders(typ=None, alg='RS256', jwk=JWK(e='AQAB', kty='RSA', n='nOC6RY5CimucaX9HjZFJPyOK-WNXGTAA_5zJgB6AAacSty5OiGBBx4ltub9YNuEaqNiSYctknZBq5JIe-lpzd2z6FcR6J45urCaKpZ6OhKaOecMWPvA91bb0uwp_W5HcH__cOKqKngV2m2st0p7VpS-UF2n8sFOsDdgBXQW3dvn73GgyWTy7DggEalJLwMYFkqQQ4LeyzGId_G3jkxFIZDSKFhfIJsrFK8mPFcFOU-e85hpDbx_3mjwwqqeYLi9YRnniRczzKFbKvrOATgYd96ihnpTC2SNj7CGHkzvJ7N2baqJPXHl2OremFofjUxZLgWuo9dWiLDTUTeT_nKHRFQ')), claims=ClientClaims(jti='test', iat=1733578607, exp=1733578622, htm='POST', htu='www.testapi.com/api/token', client_id='555'))

In [61]:
c_sig.sign(key=km.load_private_key_from_pem())

'eyJhbGciOiJSUzI1NiIsImp3ayI6eyJlIjoiQVFBQiIsImt0eSI6IlJTQSIsIm4iOiJuT0M2Ulk1Q2ltdWNhWDlIalpGSlB5T0stV05YR1RBQV81ekpnQjZBQWFjU3R5NU9pR0JCeDRsdHViOVlOdUVhcU5pU1ljdGtuWkJxNUpJZS1scHpkMno2RmNSNko0NXVyQ2FLcFo2T2hLYU9lY01XUHZBOTFiYjB1d3BfVzVIY0hfX2NPS3FLbmdWMm0yc3QwcDdWcFMtVUYybjhzRk9zRGRnQlhRVzNkdm43M0dneVdUeTdEZ2dFYWxKTHdNWUZrcVFRNExleXpHSWRfRzNqa3hGSVpEU0tGaGZJSnNyRks4bVBGY0ZPVS1lODVocERieF8zbWp3d3FxZVlMaTlZUm5uaVJjenpLRmJLdnJPQVRnWWQ5NmlobnBUQzJTTmo3Q0dIa3p2SjdOMmJhcUpQWEhsMk9yZW1Gb2ZqVXhaTGdXdW85ZFdpTERUVVRlVF9uS0hSRlEifSwidHlwIjpudWxsfQ.eyJqdGkiOiJ0ZXN0IiwiaWF0IjoxNzMzNTc4NjA3LCJleHAiOjE3MzM1Nzg2MjIsImh0bSI6IlBPU1QiLCJodHUiOiJ3d3cudGVzdGFwaS5jb20vYXBpL3Rva2VuIiwiY2xpZW50X2lkIjoiNTU1In0.RvWNQ6gFaIDBX0PkL6vqymbplslm07B9iVtM6vwrQvrG9I1agU-gTcod9hHy7DQXN7LDUVFvn6iXLrz8clHGIFKgZa-X0fJkVrj4zm7ct8alcGNfQtrlwXcv2H4LZJ9KL_Hol9hJxEdPZmisC1HpMUxg8ClHR9JqQ0KlHil67Vat-MlXWzVjLKS0HmBL-dkosk8g8d_ujVA--mOPAIIsc3_keBY54rMRDFd3BF_I45h56i7rdm7NWuu79MBWzwQrEGyoNIa6NcnGN-x-xcuxZGLwPt-IXQ6KAJxKUqm0-Z0qfn8l

In [65]:
Create(DPoPModel(**c_sig.claims.model_dump()))

HTTPException: 400: Record already exists

In [63]:
Read(DPoPModel())

,jti
0,test


In [64]:
Read(DPoPModel())

,jti
0,test


In [14]:
c_sig.model_dump()

{'headers': {'typ': 'dpop+jwt',
  'alg': 'RS256',
  'jwk': {'e': 'AQAB',
   'kty': 'RSA',
   'n': 'tbRxtdgOHdYFx0V8CY0xjFxrxREm30wdiNtz5PJ_8iKSR0Tsuad1AnN98AkgFa5SCG4VyJyFU9ogpygqIhb497Y-w6TUaGWAHpttOALdVT9rbz30XBEcgE92KNQAhvVhD1z1iam1KIiqvN_hJ1NpvFqky4W0LxS8A9L4fjLSsRMXGGOUCTX-Qm2up4iaAsKFxZMIAaggiIeIkmNXMG36EOSpkog1CHTGZ_7AOIk2RP9w7e5HqHArbGvH2aWRW8BiNlygPwd0BI8QJSxpXsvjKgMw7DsNdkrjWOdPBZpQodpTSnjesKflLF3vnYQqqOSewK2Mc-c9kl7s-GorIfHnmw'}},
 'claims': {'jti': 'test',
  'iat': 1733491282,
  'exp': 1733491297,
  'htm': 'GET',
  'htu': 'www.testapi.com/api/token',
  'client_id': '555'}}

In [15]:
valid_signature = ClientSignature.verify_signature(signature=signature)
valid_signature

ClientSignature(headers=ClientHeaders(typ='dpop+jwt', alg='RS256', jwk=JWK(e='AQAB', kty='RSA', n='tbRxtdgOHdYFx0V8CY0xjFxrxREm30wdiNtz5PJ_8iKSR0Tsuad1AnN98AkgFa5SCG4VyJyFU9ogpygqIhb497Y-w6TUaGWAHpttOALdVT9rbz30XBEcgE92KNQAhvVhD1z1iam1KIiqvN_hJ1NpvFqky4W0LxS8A9L4fjLSsRMXGGOUCTX-Qm2up4iaAsKFxZMIAaggiIeIkmNXMG36EOSpkog1CHTGZ_7AOIk2RP9w7e5HqHArbGvH2aWRW8BiNlygPwd0BI8QJSxpXsvjKgMw7DsNdkrjWOdPBZpQodpTSnjesKflLF3vnYQqqOSewK2Mc-c9kl7s-GorIfHnmw')), claims=ClientClaims(jti='test', iat=1733491282, exp=1733491297, htm='GET', htu='www.testapi.com/api/token', client_id='555'))

In [16]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="GET", endpoint=htu)
except Exception as e:
    print(e)

In [17]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="POST", endpoint=htu)
except Exception as e:
    print(e)

400: http method mismatched


In [18]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="GET", endpoint="www.wrongendpoint/api/token")
except Exception as e:
    print(e)

400: http endpoiont mismatched


In [19]:
try:
    ClientSignature.verify_signature(signature=signature).claims.validate_method_endpoint(method="POST", endpoint="www.wrongendpoint/api/token")
except Exception as e:
    print(e)

400: http method and endpoint mismatched


In [20]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [21]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())
jkt

JKT(jkt='s7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU')

In [22]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id, cnf=jkt)
access_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [23]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491283,
 'exp': 1733491883,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

In [24]:
ServerSignature.verify_signature(
    signature=access_token, 
    key=skm.load_public_key_from_pem()
).model_dump()

{'jti': 'test',
 'iat': 1733491283,
 'exp': 1733491883,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

In [25]:
jti = "test"
iat = int(time.time())
exp = iat + REFRESH_TOKEN_LIVE
client_id = "555"

In [26]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id)
refresh_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [27]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491284,
 'exp': 1733492724,
 'client_id': '555',
 'cnf': None}

In [28]:
ServerSignature.verify_signature(
    signature=refresh_token, 
    key=skm.load_public_key_from_pem()
).model_dump()

{'jti': 'test',
 'iat': 1733491284,
 'exp': 1733492724,
 'client_id': '555',
 'cnf': None}

# DPoP Signature Verification Failed (Tampered)

In [29]:
jti = "test"
iat = int(time.time())
exp = iat + DPOP_LIVE
htm = "GET"
htu = "www.testapi.com/api/token"
client_id = "555"

In [30]:
c_sig = ClientSignature(
    headers=ClientHeaders.from_key(key=km.load_public_key_from_pem()),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm=htm, htu=htu, client_id=client_id)
)
c_signature = c_sig.sign(key=km.load_private_key_from_pem())

In [31]:
fc_sig = ClientSignature(
    headers=ClientHeaders.from_key(key=km.load_public_key_from_pem()),
    claims=ClientClaims(jti=jti, iat=iat, exp=exp, htm="POST", htu=htu, client_id=client_id)
)
fc_signature = fc_sig.sign(key=km.load_private_key_from_pem())

In [32]:
h, _, s = c_signature.split(".")
_, c, _ = fc_signature.split(".")

try:
    ClientSignature.verify_signature(signature=".".join([h,c,s]))
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


# ACCESS TOKEN Signature Verification Failed (Tampered)

In [33]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [34]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())

In [35]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id, cnf=jkt)
access_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [36]:
Create(AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()))

,Count
0,1


In [37]:
Read(AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()))

,jti,access_token,client_id,exp,active,remark
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491886,True,


In [38]:
Update(
    existing=AccessTokenModel(access_token=access_token, active=True, remark="", **s_sig.model_dump()),
    new=AccessTokenModel(access_token=access_token, active=False, remark="tampered"),
)

,Count
0,1


In [39]:
Read(AccessTokenModel())

,jti,access_token,client_id,exp,active,remark
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491886,False,tampered


In [40]:
fs_sig = ServerSignature(jti="fake", iat=iat, exp=exp, client_id=client_id, cnf=jkt)
f_access_token = fs_sig.sign(key=skm.load_private_key_from_pem())

In [41]:
h, _, s = access_token.split(".")
_, c, _ = f_access_token.split(".")

In [42]:
try:
    ServerSignature.verify_signature(signature=".".join([h,c,s]), key=skm.load_public_key_from_pem())
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


In [43]:
s_sig.model_dump()

{'jti': 'test',
 'iat': 1733491286,
 'exp': 1733491886,
 'client_id': '555',
 'cnf': {'jkt': 's7uyr-kkOQMZBuJJeWxyG3j4rBKF2chcZhfolIUY1AU'}}

# REFRESH TOKEN Signature Verification Failed (Tampered)

In [44]:
jti = "test"
iat = int(time.time())
exp = iat + ACCESS_TOKEN_LIVE
client_id = "555"

In [45]:
jkt = JKT.from_key(key=valid_signature.headers.jwk.to_key())

In [46]:
s_sig = ServerSignature(jti=jti, iat=iat, exp=exp, client_id=client_id)
refresh_token = s_sig.sign(key=skm.load_private_key_from_pem())

In [47]:
Create(RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, remark="", **s_sig.model_dump()))

,Count
0,1


In [48]:
Read(RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, remark="", **s_sig.model_dump()))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491890,True,,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...


In [49]:
Update(
    existing=RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=True, **s_sig.model_dump()),
    new=RefreshTokenModel(access_token=access_token, refresh_token=refresh_token, active=False, remark="tampared"),
)

,Count
0,1


In [50]:
Read(RefreshTokenModel())

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...,555,1733491890,False,tampared,eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJqdGkiO...


In [51]:
fs_sig = ServerSignature(jti="fake", iat=iat, exp=exp, client_id=client_id)
f_refresh_token = fs_sig.sign(key=skm.load_private_key_from_pem())

In [52]:
h, _, s = refresh_token.split(".")
_, c, _ = f_refresh_token.split(".")

In [53]:
try:
    ServerSignature.verify_signature(signature=".".join([h,c,s]), key=skm.load_public_key_from_pem())
except Exception as e:
    print(e)

400: Unexpected error: Signature verification failed.


In [1]:
from fastapi import HTTPException

In [134]:
import base64
import hashlib
import os

def generate_code_verifier():
    """Generate a random code verifier."""
    return base64.urlsafe_b64encode(os.urandom(32)).rstrip(b'=').decode('utf-8')

def generate_code_challenge(code_verifier):
    """Generate a code challenge based on the code verifier."""
    digest = hashlib.sha256(code_verifier.encode('utf-8')).digest()
    return base64.urlsafe_b64encode(digest).rstrip(b'=').decode('utf-8')

# Example
code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)
print(f"Code Verifier: {code_verifier}")
print(f"Code Challenge: {code_challenge}")


Code Verifier: RJdFqu9s3om2g8oitWYm_bgTYw2i97V9TgMoqYflPUM
Code Challenge: JJPdWV5zBrYQgyB29vY721JVOb9MxArL3xjtLMABtJg


In [34]:
AUTHORIZATION_SERVER = "https://server.example.com"
CLIENT_ID = "s6BhdRkqt"
REDIRECT_URI = "https://client.example.com/cb"
SCOPE = "read_profile"

auth_url = (
    f"{AUTHORIZATION_SERVER}/authorize?response_type=code"
    f"&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}"
    f"&scope={SCOPE}&code_challenge={code_challenge}&code_challenge_method=S256"
)

print(f"Visit this URL to log in: {auth_url}")


Visit this URL to log in: https://server.example.com/authorize?response_type=code&client_id=s6BhdRkqt&redirect_uri=https://client.example.com/cb&scope=read_profile&code_challenge=aUdMQ-V4vLWdMAEG2UHBknz8NSMx-ZvPY48F9-eGeM8&code_challenge_method=S256


In [1]:
from package.database_management.data_models import UserModel

In [2]:
import pandas as pd
user_data = pd.DataFrame({
    "client_id": ["555"],
    "username": ["555"],
    "password": ["555"],
})

In [8]:
user_data.iloc[0].to_dict()

{'client_id': '555', 'username': '555', 'password': '555'}

In [9]:
UserModel(**user_data.iloc[0].to_dict())

UserModel(client_id='555', username='555', password='555')

In [13]:
import base64
import os

# Generate 16 random bytes
random_bytes = os.urandom(16)

# Encode in URL-safe Base64
token = base64.urlsafe_b64encode(random_bytes).rstrip(b'=').decode('utf-8')
print(token)


hTpzPpSngbBsOoo0QORCdg


In [14]:
import hashlib
import os

# Generate 16 random bytes
random_bytes = os.urandom(16)

# Create SHA256 hash
hash_object = hashlib.sha256(random_bytes)
hashed_token = base64.urlsafe_b64encode(hash_object.digest()).rstrip(b'=').decode('utf-8')
print(hashed_token)


omMgQaCU-oZk0D502Si5P8UxVj3Pp2QU3lDPMQ_SUsA


In [15]:
import secrets
import base64
import os
import hashlib

# 1. Using secrets.token_urlsafe(16)
secrets_token = secrets.token_urlsafe(16)
print(f"secrets.token_urlsafe(16): {secrets_token}")

# 2. Using base64 with os.urandom(16)
random_bytes = os.urandom(16)
base64_token = base64.urlsafe_b64encode(random_bytes).rstrip(b'=').decode('utf-8')
print(f"base64 with os.urandom(16): {base64_token}")

# 3. Using hashlib (SHA256) with os.urandom(16)
hash_object = hashlib.sha256(random_bytes)
hashed_token = base64.urlsafe_b64encode(hash_object.digest()).rstrip(b'=').decode('utf-8')
print(f"hashlib SHA256 (with os.urandom(16)): {hashed_token}")

# ----------- Encoding and Decoding -----------

# Decode the base64 string from the second method
decoded_base64 = base64.urlsafe_b64decode(base64_token + '==')  # Adding padding
print(f"Decoded base64 token (original random bytes): {decoded_base64}")

# Encode back the decoded bytes to check if we can get the same result
re_encoded_base64 = base64.urlsafe_b64encode(decoded_base64).rstrip(b'=').decode('utf-8')
print(f"Re-encoded base64 token (to match): {re_encoded_base64}")

# Test decoding the secrets token (no need for padding)
decoded_secrets_token = base64.urlsafe_b64decode(secrets_token + '==')
print(f"Decoded secrets token: {decoded_secrets_token}")

# Test decoding the hashed token (hashed tokens can't be decoded back, this is just for demonstration)
try:
    decoded_hashed_token = base64.urlsafe_b64decode(hashed_token + '==')
    print(f"Decoded hashed token: {decoded_hashed_token}")
except Exception as e:
    print(f"Error decoding hashed token: {e}")



secrets.token_urlsafe(16): WLcKHVU9o1in9CHlje6bOw
base64 with os.urandom(16): bU6iYZDXUgCmSVFX8P4qKA
hashlib SHA256 (with os.urandom(16)): 7v4TSmqCLJj_uY1ePiZR4_IjZOow2PMDq_qKsav-Umc
Decoded base64 token (original random bytes): b'mN\xa2a\x90\xd7R\x00\xa6IQW\xf0\xfe*('
Re-encoded base64 token (to match): bU6iYZDXUgCmSVFX8P4qKA
Decoded secrets token: b'X\xb7\n\x1dU=\xa3X\xa7\xf4!\xe5\x8d\xee\x9b;'
Decoded hashed token: b'\xee\xfe\x13Jj\x82,\x98\xff\xb9\x8d^>&Q\xe3\xf2#d\xea0\xd8\xf3\x03\xab\xfa\x8a\xb1\xab\xfeRg'


In [144]:
import base64
import hashlib
import random
import string
import os

def generate_code(seed=None):
    """Generate a code (e.g., an authorization code for OAuth2)."""
    if seed is not None:
        random.seed(seed)  # Freeze the randomness with a fixed seed
    return ''.join(random.choices(string.ascii_letters + string.digits, k=32))

def generate_code_verifier(seed=None):
    """Generate a random code verifier (frozen with a seed)."""
    if seed is not None:
        random.seed(seed)  # Freeze the randomness with a fixed seed
    return ''.join(random.choices(string.ascii_letters + string.digits + '-._~', k=32))

def generate_code_challenge(code_verifier):
    """Generate a code challenge based on the code verifier."""
    digest = hashlib.sha256(code_verifier.encode('utf-8')).digest()
    return base64.urlsafe_b64encode(digest).rstrip(b'=').decode('utf-8')

# Example Usage
seed_value = 42  # Fixed seed for predictable randomness

# Generate code, code_verifier, and code_challenge with the frozen randomness
code = generate_code(seed=seed_value)
code_verifier = generate_code_verifier(seed=seed_value)
code_challenge = generate_code_challenge(code_verifier)

print(f"Code: {code}")
print(f"Code Verifier: {code_verifier}")
print(f"Code Challenge: {code_challenge}")


Code: NbrnTP3fAbnFbmOHnKYaXRvj7uff0LYT
Code Verifier: QbsoWS6fBboHbnQJoM1a1Uwk.wgg3N1W
Code Challenge: OBvjDXn_CEItqII6NfOdXYIiMwQYFsg2NwVtZ2x2eQk


In [163]:
from package.ezorm.crud import Read
from package.database_management.data_models import UserModel, CodeModel
Read(UserModel())

,client_id,username,password
0,testclientid,testusername,testpassword


In [164]:
Read(CodeModel())

,client_id,code_challenge,code
